# CS 4501 Algorithmic Economics - Project 1

**Note:** For each of the question, please add some print or graph drawing commands to show your results in a clear way and also necessary analyses and demonstrations to help people who are not in your group understand your logics and results.


By: Param Damle, Kelvin Peng, and Sidhardh Burre

**Overview** 

We first create customer feature vectors from the given dataset. Using each customer's feature vector in the matrix X, we then train a beta and alpha matrix to predict a given customer's utility for buying any one of the products.

Within the process of Part 1, there were two significant tradeoffs that we made. The first tradeoff was that due to the sparsity of our dataset, we were forced to use a low ratio threshold to determine utility. Specifically, we found that setting the threshold for positive utility (u = 1) to be a purchase every 100 transactions and 0 otherwise. If we did not make this tradeoff, we would be subject to the fact that many of our products would have 0 utility. Conversely, if our utility threshold was set lower, buying the product at any time with any frequency (or lack thereof) would give us an indicator for positive utility. 

The other tradeoff that we were forced to make was the approximation of the hyperparameters for each product. We originally planned to train parameters for each and every item on the raw matrix of ~200 width and ~2,000,000 entries. But due to computational constraints, we decided that it would be more feasible to train the model on a PCA'd version of our matrix. We finalized on PCAing our input vectors from 200 length vectors down to 50 with a 16% loss rate.

Part 2: Online learning basically means that we start out with a bunch of random consumers with uniform distributions of preferences (non-inclined to pick any product). Each customer in this set selects a product and the customer is informed of the utility product (with some added noise). THe customer either increases or decreases the chance of selecting this product. After a certain number of rounds of this is completed, we find what any given customer’s maximum utility is for an item. If a banana was purchased their utility for the banana is 1. The difference between that and the average utility for each consumer after running the simulation. Take the gap and then average. How far were the customer’s utilities at the end from the highest utility possible. 

## Part 1

### Question 1
Using a Jupyter notebook import the csv file as pandas dataframe.

In [ ]:
import pickle
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
from google.colab import drive

import scipy as scp
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA

import seaborn as sns

import statsmodels.api as sm
import matplotlib.pyplot as plt
import math

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
drive.mount('/content/drive')

project_dir = "/content/drive/MyDrive/algo_econ_best_group/project1/"
# project_dir = "/content/drive/MyDrive/Sem 4/CS 4501 Algo Econ/algo_econ_best_group/project1/"

Mounted at /content/drive


In [ ]:
df = pd.read_csv (project_dir + "scanner_data.csv")
df.drop(df.columns[0], axis=1, inplace=True)
print (df)

num_customers = len(pd.unique(df['Customer_ID']))
num_products = len(pd.unique(df['SKU']))
num_categories = len(pd.unique(df['SKU_Category']))

print(  "Number of Customer_ID\t",   num_customers   )
print(  "Number of Products   \t",   num_products   )
print(  "Number of Categories \t",   num_categories   )

              Date  Customer_ID  Transaction_ID SKU_Category    SKU  Quantity  \
0       02/01/2016         2547               1          X52  0EM7L       1.0   
1       02/01/2016          822               2          2ML  68BRQ       1.0   
2       02/01/2016         3686               3          0H2  CZUZX       1.0   
3       02/01/2016         3719               4          0H2  549KK       1.0   
4       02/01/2016         9200               5          0H2  K8EHH       1.0   
...            ...          ...             ...          ...    ...       ...   
131701  04/07/2016        20203           32900          IEV  FO112       3.0   
131702  04/07/2016        20203           32900          N8U  I36F2       1.0   
131703  04/07/2016        20203           32900          U5F  4X8P4       1.0   
131704  04/07/2016        20203           32900          0H2  ZVTO4       1.0   
131705  04/07/2016        20203           32900          Q4N  QM9BP       1.0   

        Sales_Amount  
0   

### Question 2
The fact that consumer does not purchase anything can be 
interpreted as that she chose an outside option. Given that the fact that she chose an outside option is not recorded in this dataset, argue how you would construct a proxi variable for the choice of an outside option. Add such a proxi variable to your dataframe. 

**Hint:** you can use information that some consumers do not appear in the data every week.

**Answer:**

The problem here is to essentially analyze the attendance of customers and generate an “absent visit” element from that analysis. There are two ways to go about this. The more analytically accurate way to do this is to find the visit pattern for every customer and then insert the proxy variable stating that the customer did not visit in a manner that is consistent with their pattern. 

The former approach is difficult so instead we assumed that a normal customer would come to the store on a weekly basis and inserted a proxy variable based off of that. 

But, we do not use this new dataset in the rest of our analysis. The new dataset has 1 million entries whereas the original dataset only contained ~100,000 entries. If we were forced to use the older dataset, our analysis would’ve taken an inordinate amount of time. 

We take into account attendance in a different manner by calculating the weekly frequency with which the customer comes to the store and incorporating this value into the customer’s feature vector. 


In [ ]:
"""
    Process dataframe and add proxy variable
"""

consumer_set = set(df["Customer_ID"]) # set of all unique customer IDs

df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y') # parse date strings to date objects

start_date = min(df['Date']) # first date in dataset
end_date = max(df['Date']) # last date in dataset
num_weeks = (end_date - start_date).days // 7 # number of weeks in dataset
print("This dataset spans from " + str(start_date) + " to " + str(end_date) + ". That's " + str(num_weeks) + " weeks.")

weeks = {}  # maps start date of each week to np array of customers who bought stuff that week

# map each week to the set of consumers who bought during that week
for index, row in df.iterrows():
    w = (row['Date'] - start_date).days // 7
    date = start_date + w * timedelta(weeks=1)
    if not date in weeks:
        weeks[date] = np.array([])
    np.append(weeks[date], row['Customer_ID'])

arr = np.empty((0,7)) # feature matrix data (with proxy)
t = len(df["Transaction_ID"]) # transaction ID counter

# for every week and for all customers, if customer did not buy within week add a proxy transaction
for w in weeks:
    weeks_consumer = set(weeks[w]) # consumers whom appear in the week
    diff = set(consumer_set - weeks_consumer) # difference
    arr = np.append(arr, [[w, int(c), int(t_id), "PROXY", "PROXY", float(1), float(0)] for c, t_id in zip(diff, range(t, t + len(diff)))], axis=0)
    t += len(diff)

df_proxy = pd.DataFrame(arr, columns=df.columns) # proxy rows added
df_proxied = pd.concat([df, df_proxy]).astype(df.dtypes.to_dict()) # dataframe with proxy rows added

print(df_proxied)

This dataset spans from 2016-01-02 00:00:00 to 2016-12-31 00:00:00. That's 52 weeks.
              Date  Customer_ID  Transaction_ID SKU_Category    SKU  Quantity  \
0       2016-01-02         2547               1          X52  0EM7L       1.0   
1       2016-01-02          822               2          2ML  68BRQ       1.0   
2       2016-01-02         3686               3          0H2  CZUZX       1.0   
3       2016-01-02         3719               4          0H2  549KK       1.0   
4       2016-01-02         9200               5          0H2  K8EHH       1.0   
...            ...          ...             ...          ...    ...       ...   
1199120 2016-12-31        22621         1330826        PROXY  PROXY       1.0   
1199121 2016-12-31        22622         1330827        PROXY  PROXY       1.0   
1199122 2016-12-31        22623         1330828        PROXY  PROXY       1.0   
1199123 2016-12-31        22624         1330829        PROXY  PROXY       1.0   
1199124 2016-12-31      

### Question 3
Given that we do not have **explicit** consumer feature vectors $\mathbf{x}^i = (x^i_1, \cdots, x_k^i)$  in the data, discuss how you would construct such feature vectors for each consumer $i$ from the given data. Add your constructed characteristics to your dataframe. 

**Hint:** you can use transaction history and argue that past shopping patterns may give a good characterization for a given consumer.

**Answer:**

To generate the vectors we first considered what metrics would be representative of the customer’s behavior. Eventually we decided on the following metrics: 

*   Average total transactions per week (number of shopping trips/week)
*   Average number of items purchased per week
*   Average number of different items / transactions
*   Average price of product purchased per product category (0 if category was never purchased)

The reason these features were selected was for the following reasons. We needed efficiently computable, highly generalizable features such that any analysis would be easily applicable to other customers as well. Most of these features are averages with a common denominator. 

We felt that the first three were relevant features were relevant on the basis of shopping game-theory. Consider the following question: 

**"What does it take for someone to visit the grocery store?"**

This question leads into many of the implied assumptions that we make in selecting our features. By considering the above question, we concluded that people visited the store and made purchases only when the cumulative sum of the utility of all the products purchased during a single transaction was positive (accounting for sunk exploration and transportation costs). Therefore, we found it relevant to create features based on a boundary threshold for which the shopper would come to the store. To put it another way, we wanted to use each consumer's shopping trip sunk cost in our model.

Thus, our feature vector uses some features based on the frequency of transactions and items purchased per transaction.

Our "average price of product purchased per product category" feature is a series of 186 averages, one for each SKU category. The feature captures the relative price of a customer's purchased products, compared to the other available products within the SKU category. The reason this is so important is because it implicitly offers a notion of the customer’s quality preference. Purchasing more expensive items within the SKU category is a clear indication that the customer highly values that item and makes a conscious decision to purchase the more expensive alternative over the cheaper. Based on the principles of the income and substitution effects, we empirically capture consumers' wealth levels and quality preference.

In [ ]:
df = df_proxied   # WOULD DO THIS, but we cannot afford to pay for extra Google Colab RAM.
# # this increases X size by tenfold, which is unnecessary.
# df.head()

In [ ]:
customers = sorted(list(df['Customer_ID'].unique()))
products = sorted(list(df['SKU'].unique()))
categories = sorted(list(df['SKU_Category'].unique()))

In [ ]:
sku_to_index = {} # dict to map SKU string to index in products list
for ic, c in enumerate(products):
  sku_to_index[c] = ic

In [ ]:
customer_to_index = {} # map customer ID to index in customers list
for ic, c in enumerate(customers):
  customer_to_index[c] = ic

In [ ]:
products_per_category = {}

for SKU_cat in pd.unique(df['SKU_Category']):
    products_per_category[SKU_cat] = set()
    bool_vec_SKU_cat = df['SKU_Category'] == SKU_cat
    rows = df.loc[bool_vec_SKU_cat]
    for SKU in pd.unique(rows['SKU']):
        products_per_category[SKU_cat].add(SKU)

# get SKU category for a given SKU
def get_SKU_category(SKU):
    for SKU_cat in products_per_category:
        if SKU in products_per_category[SKU_cat]:
            return SKU_cat
    return None

In [ ]:
# PICKLE DUMP

'''
fprice1 = open(project_dir + "prices", 'wb')
pickle.dump(prices, fprice1)
fprice1.close()
'''

'\nfprice1 = open(project_dir + "prices", \'wb\')\npickle.dump(prices, fprice1)\nfprice1.close()\n'

In [ ]:
# PICKLE LOAD

fprice2 = open(project_dir + "prices", 'rb')
prices = pickle.load(fprice2)
fprice2.close()

In [ ]:
transactions_per_customer = {}  # maps customer to set of transaction ids
transactions_per_week = {}  # maps customer to average number of transactions per week
quantity_per_week = {}  # maps customer to average quantity of all products bought per week
transaction_diversity = {}  # maps transaction id to number of rows it has in df
diversity_per_transaction = {}  # maps customer id to average number of different items per transaction

spending_per_customer_per_category = {}  
# maps (customer, SKU Category) to (how much customer spent on this category, how much quantity customer bought in this category)


for index, row in df.iterrows():
  if index%13000 == 0:
    print(str(int(index*100.0/df.shape[0])) + "%")
  if row['SKU'] != 'PROXY':
    cust = row['Customer_ID']
    category = row['SKU_Category']
    t_id = row['Transaction_ID']

    if cust not in transactions_per_customer:  # new customer
      transactions_per_customer[cust] = set()
      quantity_per_week[cust] = 0
      transactions_per_week[cust] = 0
    transactions_per_customer[cust].add(t_id)

    quantity_per_week[cust] += row['Quantity'] * 1.0 / num_weeks

    if t_id not in transaction_diversity:  # new transaction
      transaction_diversity[t_id] = 0
      transactions_per_week[cust] += 1.0 / num_weeks
    transaction_diversity[t_id] += 1

    if (cust, category) not in spending_per_customer_per_category:
      spending_per_customer_per_category[(cust, category)] = (0, 0)
    prev_spcpc = spending_per_customer_per_category[(cust, category)]
    spending_per_customer_per_category[(cust, category)] =  (prev_spcpc[0] + row['Sales_Amount'], prev_spcpc[1] + row['Quantity'])

for tempc in customers:
  total_trans = 0
  for trans in transactions_per_customer[tempc]:
    total_trans += transaction_diversity[trans]
  diversity_per_transaction[tempc] = total_trans * 1.0 / len(transactions_per_customer[tempc])


0%
0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
0%
0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
84%
85%
86%
87%
88%
89%


In [ ]:
# PICKLE DUMP

# f0 = open(project_dir + "dpt", 'wb')
# pickle.dump(diversity_per_transaction, f0)
# f0.close()
# f1 = open(project_dir + "spcpc", 'wb')
# pickle.dump(spending_per_customer_per_category, f1)
# f1.close()

In [ ]:
# PICKLE LOAD

# f2 = open(project_dir + "dpt", 'rb')
# diversity_per_transaction = pickle.load(f2)
# f2.close()
# f3 = open(project_dir + "spcpc", 'rb')
# spending_per_customer_per_category = pickle.load(f3)
# f3.close()

In [ ]:
#restucture your data
xsize = 191
X_customers = np.zeros((num_customers, xsize))

print(spending_per_customer_per_category)
print(spending_per_customer_per_category[2547, 'X52'])

for i_customer, customer in enumerate(customers):
  X_customers[i_customer][0] = transactions_per_week[customer]
  X_customers[i_customer][1] = quantity_per_week[customer]
  X_customers[i_customer][2] = diversity_per_transaction[customer]

  for i_cat_temp, category in enumerate(categories):
    i_category = i_cat_temp + 3
    if (customer, category) in spending_per_customer_per_category:
      X_customers[i_customer][i_category] = spending_per_customer_per_category[(customer, category)][0]/spending_per_customer_per_category[(customer, category)][0]

{(2547, 'X52'): (16.24, 4.0), (822, '2ML'): (5.46, 1.0), (3686, '0H2'): (30.21, 4.0), (3719, '0H2'): (14.24, 2.0), (9200, '0H2'): (6.88, 1.0), (5010, 'JPI'): (10.77, 1.0), (1666, 'XG4'): (28.770000000000003, 5.1), (1666, 'FEW'): (12.75, 2.0), (1253, '0H2'): (13.84, 2.0), (5541, 'N5F'): (8.18, 1.0), (5541, 'H8O'): (6.35, 1.0), (7548, 'N8U'): (2.11, 1.0), (7548, 'JR5'): (5.38, 1.0), (6865, 'I4Y'): (8.04, 1.0), (6044, 'N8U'): (3.62, 1.0), (6044, 'X52'): (3.96, 1.0), (592, 'YMJ'): (11.65, 1.0), (592, 'P42'): (9.0, 1.0), (6450, 'N8U'): (7.02, 1.0), (3874, 'FEW'): (27.060000000000002, 2.0), (3874, 'SFC'): (14.25, 6.0), (5599, '7C6'): (37.209999999999994, 4.0), (5599, 'OQA'): (25.74, 2.0), (9223, 'TW8'): (1.62, 1.0), (9223, 'Q4N'): (10.5, 1.0), (4921, 'H15'): (3.75, 1.0), (6294, 'N5F'): (5.82, 2.0), (6294, 'F9B'): (7.9, 1.0), (6294, 'C8Z'): (46.230000000000004, 4.0), (5780, 'BR0'): (22.25, 1.0), (2116, 'X52'): (7.02, 1.0), (2116, '2ML'): (7.29, 1.0), (3624, 'P42'): (5.87, 1.0), (1253, 'R6E'):

In [ ]:
print(df)

              Date  Customer_ID  Transaction_ID SKU_Category    SKU  Quantity  \
0       2016-01-02         2547               1          X52  0EM7L       1.0   
1       2016-01-02          822               2          2ML  68BRQ       1.0   
2       2016-01-02         3686               3          0H2  CZUZX       1.0   
3       2016-01-02         3719               4          0H2  549KK       1.0   
4       2016-01-02         9200               5          0H2  K8EHH       1.0   
...            ...          ...             ...          ...    ...       ...   
1199120 2016-12-31        22621         1330826        PROXY  PROXY       1.0   
1199121 2016-12-31        22622         1330827        PROXY  PROXY       1.0   
1199122 2016-12-31        22623         1330828        PROXY  PROXY       1.0   
1199123 2016-12-31        22624         1330829        PROXY  PROXY       1.0   
1199124 2016-12-31        22625         1330830        PROXY  PROXY       1.0   

         Sales_Amount  
0  

In [ ]:
print(X_customers.shape)

(22625, 191)


**Note**

Here is where we conduct our PCA on the vectors containing the features that we determined relevant. Note the tradeoff between the number of components that we PCA down too and the amount of variance that we can explain. In this specific case, we can learn from vectors that are only a quarter of the original size while maintaining 84% of the information in the vecotrs. 

In [ ]:
pca = PCA(n_components=50)
pcafile = open(project_dir + "pca32", "wb")
pickle.dump(pca, pcafile)
pcafile.close
X_customers = pca.fit_transform(X_customers)

# pcaload = open(project_dir + "pca32", "rb")
# pca = pickle.load(pcaload)
# pcaload.close()
# X = pca.transform(X)

xsize = X_customers.shape[1]
print("PCA explains " + str(pca.explained_variance_ratio_.sum() * 100) + "% of variance")

PCA explains 83.99666457685238% of variance


In [ ]:
pca.explained_variance_ratio_

array([0.29884921, 0.05822632, 0.03877815, 0.03059783, 0.0244079 ,
       0.01940148, 0.01932298, 0.01669038, 0.01592954, 0.01536772,
       0.01502365, 0.01409735, 0.01385038, 0.0119186 , 0.01114393,
       0.01082759, 0.01043832, 0.01021942, 0.00998815, 0.0095956 ,
       0.00953583, 0.00931751, 0.00919851, 0.00902038, 0.00842766,
       0.00829525, 0.00809877, 0.00759854, 0.00752346, 0.0068157 ,
       0.00644186, 0.00639537, 0.00625566, 0.00608574, 0.00581323,
       0.00570863, 0.00523801, 0.00519339, 0.00501624, 0.00495606,
       0.0049003 , 0.00480974, 0.00466476, 0.00448419, 0.00440772,
       0.00434237, 0.00431625, 0.00425047, 0.00409516, 0.00408536])

In [ ]:
###################################### MAKE ACTUAL X HERE
# go thru df
# for each transaction containing customer i and product j
#   append X_customers[i] to price[product[j]] and append this vector to X
#   append int(product[j]) = indexof(SKU) to y

X_matrix = np.zeros((len(df), X_customers.shape[1] + 1))

Y_vector = np.zeros(len(df))

df_counter = 0

for i, row in df.iterrows(): 
  if df_counter % 13000 == 0:
    print(str(int(df_counter*100.0/len(df.index))) + "%")
  X_matrix[df_counter][0:X_customers.shape[1]] = X_customers[ customer_to_index[row['Customer_ID']] ]
  #new_row = np.append( X_customers[ customer_to_index[row['Customer_ID']]], np.asarray([row['Sales_Amount']/row['Quantity']]), axis = 0)
  #X[df_counter] = new_row

  X_matrix[df_counter][X_customers.shape[1]] = row['Sales_Amount']/row['Quantity']

  Y_vector[df_counter] = sku_to_index[row['SKU']]

  df_counter += 1



0%
0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
99%


In [ ]:
#Pickle for X_matrix and Y_vector

f0 = open(project_dir + "X_matrix", 'wb')
pickle.dump(X_matrix, f0)
f0.close()
f1 = open(project_dir + "Y_vector", 'wb')
pickle.dump(Y_vector, f1)
f1.close()


In [ ]:
#Pickle for X_matrix and Y_vector

f0 = open(project_dir + "X_matrix", 'rb')
X_matrix = pickle.load(f0)
f0.close()
f1 = open(project_dir + "Y_vector", 'rb')
Y_vector = pickle.load(f1)
f1.close()

In [ ]:
print(X_matrix)
print(X_matrix.shape)

[[ 5.11455906e-01  7.32934399e-01  1.13398267e+00 ... -7.10722716e-02
  -3.39275137e-02  3.13000000e+00]
 [-3.14426866e-01 -1.24522769e-01 -5.31493879e-02 ... -2.53195177e-01
  -5.95146198e-02  5.46000000e+00]
 [ 2.18800088e+00  2.28767074e+00 -1.98730653e-01 ...  3.35810930e-02
  -2.94862816e-01  6.35000000e+00]
 ...
 [-3.60719719e-02 -4.54183139e-01 -1.85758514e-01 ...  9.30646197e-02
   5.43393092e-01  0.00000000e+00]
 [-3.29474776e-02 -4.16408264e-01  2.04320417e-01 ... -1.16362860e-02
   3.23898961e-03  0.00000000e+00]
 [ 3.46190302e-01  1.29456351e-01  6.83891045e-01 ...  4.56813813e-02
   1.66847051e-01  0.00000000e+00]]
(1330831, 51)


In [ ]:
'''
pca = PCA(n_components=32)
pcafile = open(project_dir + "pca32", "wb")
pickle.dump(pca, pcafile)
pcafile.close
X = pca.fit_transform(X_matrix)

# pcaload = open(project_dir + "pca32", "rb")
# pca = pickle.load(pcaload)
# pcaload.close()
# X = pca.transform(X)

xsize = X.shape[1]
print("PCA explains " + str(pca.explained_variance_ratio_.sum() * 100) + "% of variance")

'''

'\npca = PCA(n_components=32)\npcafile = open(project_dir + "pca32", "wb")\npickle.dump(pca, pcafile)\npcafile.close\nX = pca.fit_transform(X_matrix)\n\n# pcaload = open(project_dir + "pca32", "rb")\n# pca = pickle.load(pcaload)\n# pcaload.close()\n# X = pca.transform(X)\n\nxsize = X.shape[1]\nprint("PCA explains " + str(pca.explained_variance_ratio_.sum() * 100) + "% of variance")\n\n'

### Question 4
Produce the utility parameters $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$ for every product $j$  by estimating a multinomial 
logit model from your constructed dataset.

**Answer:**
To produce the utility parameters via the multinomial logit model, we started by using the SGDClassifier function on a per-chunk basis. Because we possessed a very limited amount of computing power, we were forced to train our model one chunk at a time. From this, we gained beta, utility parameters as well as alphas for each product j. 

Prior to seeing the TA's guide on how to go about this, we attempted to create feature vectors for each customer (using the same parameters listed above) and correlate those feature vectors to the utility that the customer would recieve from purchasing the item. This model was trained on a per-customer, per item basis but proved very computationally demanding.

Training was done in chunks of 20000 rows at a time in order to make training possible at all, as loading the entire feature vector into RAM was impossible. The entire dataset was used in the training process - this was done to ensure that the model could train properly, as the data was very sparse. With a train-test split, the model could train on a set without any purchases of certain products, which would induce the training to fail. Thus, we were forced to use the entire dataset to train.

In [ ]:
type(Y_vector)

numpy.ndarray

In [ ]:
# standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# scaler.fit X matrix
scaler_X = scaler.fit(X_matrix)
scaled_X = scaler_X.transform(X_matrix)

In [ ]:
from sklearn.linear_model import SGDClassifier
from tqdm.notebook import tqdm

chunksize = 20000
clf = SGDClassifier(alpha = 0.0001, loss='log', penalty='l2', n_jobs=-1, shuffle=False, fit_intercept=False)
for i in range(int((len(X_matrix)/chunksize)+1)):
  train_X = scaled_X[i*chunksize : (i+1)*chunksize ]
  train_Y = Y_vector[i*chunksize : (i+1)*chunksize ]
  clf.partial_fit(train_X, train_Y, classes=range(num_products + 1))

In [ ]:
finished = open(project_dir + "clf_0", 'wb')
pickle.dump(clf, finished)
finished.close()

In [ ]:
pcaload = open(project_dir + "clf_0", "rb")
clf = pickle.load(pcaload)
pcaload.close()

In [ ]:
# item i of below list contains vector of betas for product i
clf.coef_

array([[ 7.01768821e-02,  7.53726595e-02,  1.40048534e-02, ...,
        -4.79805832e-03,  1.38501802e-03,  2.43974020e+01],
       [ 7.01768821e-02,  7.53726594e-02,  1.40048535e-02, ...,
        -4.79805834e-03,  1.38501802e-03,  2.43974019e+01],
       [ 7.01768822e-02,  7.53726593e-02,  1.40048535e-02, ...,
        -4.79805837e-03,  1.38501801e-03,  2.43974019e+01],
       ...,
       [ 7.01768811e-02,  7.53726604e-02,  1.40048528e-02, ...,
        -4.79805801e-03,  1.38501812e-03,  2.43974023e+01],
       [ 7.01768820e-02,  7.53726595e-02,  1.40048534e-02, ...,
        -4.79805830e-03,  1.38501803e-03,  2.43974020e+01],
       [ 7.01768809e-02,  7.53726605e-02,  1.40048526e-02, ...,
        -4.79805796e-03,  1.38501813e-03,  2.43974024e+01]])

## Part 2

### Question 1
Construct a multi-armed bandit algorithm such that

1. It is randomly initialized at first and selects **one** product out of $j$ available products.
2. It updates  $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$  over  time by observing the utility $\widehat{u}_{ij}$ of each product $j$ it selected in the past and selects new products

(See Question 2)

### Question 2

 Draw 1000 random consumers from your data. For each consumer,  run your online learning algorithm for 100 steps. Note that this is a simulation process --- i.e., your algorithm itself does not know $\beta_{0j}, \beta_{1j},\cdots \beta_{kj}$ and $\alpha_j$, but can only observe the $\widehat{u}_{ij}$ for any product $j$ that the algorithm pulled (i.e., purchased).     
 For each randomly picked consumer $i$, compute the difference $\Delta_i$ between the  maximum utility $\max_j\widehat{u}_{ij}$ (i.e., consumer $i$'s  utility for her  favorite product) and the average utility that your algorithm
achieved at the 100th step. Compute the average of $\Delta_i$ over those 1000 consumers, and explain why there is such a difference.  

**Answer**

We constructed an epsilon-greedy multi-armed bandit algorithm using a batch learning approach.

First, 1000 consumers were selected to be in the simulation. Their feature vectors were concatenated for use later. A (1000 x # products) matrix was constructed to store each conusmer's accumulated utility (total observed utility of given action) for buying each product.

Then for each timestep (100 total), we compute each consumer's probability/utility vector with the exponential probability function using their accumulated utility vector. The resultant probability vector is transformed using the epsilon to even out the probabilities, making the probabilities reflect an epsilon-greedy (rather than purely greedy) strategy.

Then, the consumer's choice SKU is randomly selected using the epsilon-greedy probability vector. Epsilon-greedy is used rather than a purely greedy strategy to encourage exploration of the action space to find other optima, rather than overly-exploiting the action with highest probable utility. With a higher epsilon, more exploration could be encouraged over exploitation and vice versa.

Utility of the consumer's SKU choice is then predicted using the beta and alpha parameters of the trained model. This "observed utility" of the SKU choice is then accumulated in the accumulated utilities matrix.

This learning framework is an example of reinforcement learning - the consumers' preferences (modeled by accumulated utilities) are product-agnostic. However, through the simulation and learning process of accumulating utility realizations the consumers' "develop" preferences.

After each iteration of the simulation, we perform batch learning using the result of each consumer's choice in the iteration. The partial fit we perform at each iteration of the simulation is exactly like an online learning model, where learning is done in piecemeal fashion in conjuction with the evaluation process.

In [ ]:
# SIMULATION

# get 1000 random indices of X (i.e. consumer IDs) without replacement
consumers_in_trial = 1000
rand = np.array(range(0, len(X_customers)))
np.random.shuffle(rand)
rand = rand[:consumers_in_trial]
accumulated_utility = np.zeros((consumers_in_trial, num_products))

epsilon = 0.05 # for epsilon-greedy

# construct array of randomly chosen consumers' feature vectors
consumers = np.empty((0,X_customers.shape[1]))
for r in rand:
    consumers = np.append(consumers, np.asarray([X_customers[r]]), axis=0)

# run simulation for 100 steps
for timestep in range(100):
    if timestep % 5 == 0:
        print(str(timestep)+"%")
    
    timestep_vectors = np.empty((0,X_customers.shape[1]+1))
    timestep_y = np.empty((0))

    # compute and accumulate timestep vectors
    for consumer_i in range(consumers_in_trial):

        # compute probability vector
        if accumulated_utility[consumer_i].sum() > 0:
            exponential = math.e**(accumulated_utility[consumer_i] / accumulated_utility[consumer_i].sum())
        else:
            exponential = np.ones((accumulated_utility.shape[1]))
            
        p = (1 - epsilon) * exponential / (exponential.sum())

        q = np.ones(exponential.shape[0]) * (epsilon * 1.0/exponential.shape[0])

        probabilities = p + q
        oldsum = probabilities.sum()
        probabilities /= probabilities.sum()

        if np.count_nonzero(np.isnan(probabilities)):
            print("SUM WAS " + str(oldsum))

        # predict consumer's choice
        chosen_SKU = np.random.choice(products, p=probabilities)

        consumer_X = np.asarray([np.append(consumers[consumer_i], np.asarray([prices[chosen_SKU]]))]) # consumer's row in X appended w/ product price
        consumer_X = scaler_X.transform(consumer_X)

        observed_utility = np.dot(clf.coef_[sku_to_index[chosen_SKU]], consumer_X[0])
        accumulated_utility[consumer_i][sku_to_index[chosen_SKU]] += observed_utility

        timestep_vectors = np.append(timestep_vectors, consumer_X, axis=0)
        timestep_y = np.append(timestep_y, np.asarray([sku_to_index[chosen_SKU]]), axis=0)

    ### ONLINE LEARNING HERE
    clf.partial_fit(timestep_vectors, timestep_y, classes=range(num_products + 1))

0%
5%
10%
15%
20%
25%
30%
35%
40%
45%
50%
55%
60%
65%
70%
75%
80%
85%
90%
95%


We compute for each consumer the difference between their mean accumulated utility and the maximum. The average discrepancy over the 1000 consumers was 0.422, which was a large difference. We interpret this delta as most consumers not purchasing a wide variety of products. Thus the accumulated utility vector is mostly sparse, making the mean utility lower. Some products are high attractive to consumers however, which would make their utility for those products spike. Thus there is a wide variance in maximum vs. average accumulated utility for products for each consumer.

In [ ]:
delta_i = []
for consumer_i in range(consumers_in_trial):
    delta_i.append(np.max(accumulated_utility[consumer_i]) - np.mean(accumulated_utility[consumer_i]))
mean_delta_i = np.mean(np.asarray(delta_i))
print("Average discrepancy between max and avg utility (delta_i): " + str(mean_delta_i))

Average discrepancy between max and avg utility (delta_i): 0.4223158895060251


In [ ]:
print(accumulated_utility)
print(accumulated_utility.sum(axis=0).mean())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
-79.01432017362926
